In [16]:
import pandas as pd
import numpy as np
from datetime import date, time, datetime
import scipy.optimize
from functools import reduce
from datetime import date

In [4]:
def xirr(df, guess=0.05, date_column='date', amount_column='amount'):
    '''Calculates XIRR from a series of cashflows. 
       Needs a dataframe with columns date and amount, customisable through parameters. 
       Requires Pandas, NumPy libraries'''

    df = df.sort_values(by=date_column).reset_index(drop=True)

    amounts = df[amount_column].values
    dates = df[date_column].values

    years = np.array(dates-dates[0], dtype='timedelta64[D]').astype(int)/365

    step = 0.05
    epsilon = 0.0001
    limit = 1000
    residual = 1

    #Test for direction of cashflows
    disc_val_1 = np.sum(amounts/((1+guess)**years))
    disc_val_2 = np.sum(amounts/((1.05+guess)**years))
    mul = 1 if disc_val_2 < disc_val_1 else -1

    #Calculate XIRR    
    for i in range(limit):
        prev_residual = residual
        residual = np.sum(amounts/((1+guess)**years))
        if abs(residual) > epsilon:
            if np.sign(residual) != np.sign(prev_residual):
                step /= 2
            guess = guess + step * np.sign(residual) * mul   
        else:
            return guess

In [2]:
def xnpv_sp(rate, values, dates):
    '''Equivalent of Excel's XNPV function.

    >>> from datetime import date
    >>> dates = [date(2010, 12, 29), date(2012, 1, 25), date(2012, 3, 8)]
    >>> values = [-10000, 20, 10100]
    >>> xnpv(0.1, values, dates)
    -966.4345...
    '''
    if rate <= -1.0:
        return float('inf')
    d0 = dates[0]    # or min(dates)
    return sum([ vi / (1.0 + rate)**((di - d0).days / 365.0) for vi, di in zip(values, dates)])

def xirr_sp(values, dates):
    '''Equivalent of Excel's XIRR function.

    >>> from datetime import date
    >>> dates = [date(2010, 12, 29), date(2012, 1, 25), date(2012, 3, 8)]
    >>> values = [-10000, 20, 10100]
    >>> xirr(values, dates)
    0.0100612...
    '''
    positives = [x if x > 0 else 0 for x in values] 
    negatives = [x if x < 0 else 0 for x in values] 
    return_guess = (sum(positives) + sum(negatives)) / (-sum(negatives))

    try:
        return scipy.optimize.newton(lambda r: xnpv_sp(r, values, dates), return_guess)
    except RuntimeError:    # Failed to converge?
        return scipy.optimize.brentq(lambda r: xnpv_sp(r, values, dates), -1.0, 1e10)

In [5]:

df = pd.DataFrame({
  'date': [date(2010, 12, 29), date(2012, 1, 25), date(2012, 3, 8)],
  'amount': [-10000, 20, 10100]
})

xirr(df)

0.010061264038085938

In [6]:
dates = [date(2010, 12, 29), date(2012, 1, 25), date(2012, 3, 8)]
values = [-1000000, 2000, 1010000]
xirr_sp(values, dates)

0.01006126516496502

In [158]:
def mdietz(d, begval, endval, start_date = None, end_date = None, date_column='date', amount_column='amount'):

    d = sorted(d, key = lambda x: x[0])
    dates = [ i for i, j in d ]

    def do_sum(acc, x): 
        return 0, acc[1] + x[1]

    def wtd_sum(acc, x):
        delta = end_date - x[0]
        weight = delta.days/total_days
        return 0, acc[1] + -1 * x[1] * weight

    # total cashflows
    total_flows  =  reduce(do_sum, d)[1]
    
    if start_date is None: start_date = dates[0]
    if end_date is None: end_date = dates[-1]

    delta = end_date - start_date
    total_days = delta.days
    print(total_days, total_flows, reduce(wtd_sum, d, (0,0))[1])
    
    #Modified Dietz
    MD = (endval - begval + total_flows) / (begval + reduce(wtd_sum, d, (0,0))[1]);
    annualizedMD = (1+MD)**(365/(total_days-1)) - 1;

    return MD, annualizedMD
    

    

In [159]:
mdietz(
    tuple(zip(
        [date(2020, 1, 31), date(2020, 2, 28), date(2020, 3, 4), date(2020, 3, 31)], 
        [-3429030, -6737065, 546250, -4512623])), 
    122504000,    
    137076700,
    date(2019, 12, 31),
    date(2020, 3, 31)
    )

91 -14132468 4467902.527472528


(0.0034671607752333107, 0.014135923482372359)

In [77]:
def rsum(acc, x):
    return acc[0] + x[0], 0
a = [(1, 2), (1, 4), (1, 6)]
print(reduce(rsum, a))
     

(3, 0)


In [76]:
a = [(1, 2), (1, 4), (1, 6)]
print(reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]), a))

(3, 12)


In [32]:
nums = [(1, 0), (2, 5), (3, 10)]
reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]), nums)

(6, 15)

In [36]:
tuple(zip([date(2010, 12, 29), date(2012, 1, 25), date(2012, 3, 8)], [-10000, 20, 10100]))

((datetime.date(2010, 12, 29), -10000),
 (datetime.date(2012, 1, 25), 20),
 (datetime.date(2012, 3, 8), 10100))